In [5]:
import logging

from Loader import fileloader_proto as fl
from Loader import dfutils
import xlwings as xw
import pandas as pd
import numpy as np
import posixpath

logger = logging.getLogger("")
logger.setLevel(logging.INFO)

month = '201905'

inifile = fl.ReadIniFile(mercado="empresas")
#defaultpath = inifile.getDataPath()
#testpath = inifile.getTestPath()
#parser = inifile.getIniFileParser()


**Procedimiento**
---

- Copiar archivos de comisiones a la carpeta de reporte de productividad
- Ejecutar Script para normalizar pesos
- En el archivo final de R. Productividad eliminar los cesados y agregar los nuevos.
- Ejecutar Script para cuadrar "reporte de productividad". Ojo: Este usa el orden de planilla que esta en el reporte final de productividad.
- Este ultimo script creara un archivo en testpath, usar ese para copiar y pegar las columnas en el reporte final de productividad
    

**Normalizar Matriz de Pesos de Comisiones**
---

Tomamos $A_{mxn}=\begin{bmatrix}a_{i,j}\end{bmatrix}$y queremos obtener
$\begin{bmatrix}\frac{a_{i,j}}{\sum_{k=1}^{n}a_{i,k}}\end{bmatrix}_{1xm}$

Hallamos el vector suma de las filas:

$A_{s}=\begin{bmatrix}\sum_{k=1}^{n}a_{i,k}\end{bmatrix}_{1xm}$

Invertimos:

$(A_{s}^{-1})_{1xm}$

Hallamos la diagonal:

\{$diag(A_{s}^{-1})\}_{mxm}$

Multiplicamos esta matriz diagonal por la matriz inicial para obtener
lo buscado:

\{$diag(A_{s}^{-1})\}_{mxm}*A_{mxn}=T\{a_{i,j}\}$

$\begin{bmatrix}*_{1}\\
 & *_{2}\\
 &  & \ddots
\end{bmatrix}*\begin{bmatrix}\Delta_{1,1} & \Delta_{1,2}\begin{array}{c}
\cdots\end{array}\\
\Delta_{2,1} & \Delta_{2,2}\begin{array}{c}
\cdots\end{array}\\
\vdots & \vdots
\end{bmatrix}=\begin{bmatrix}*_{1}\Delta_{1,1} & *_{1}\Delta_{1,2}\begin{array}{c}
\cdots\end{array}\\
*_{2}\Delta_{2,1} & *_{2}\Delta_{2,2}\begin{array}{c}
\cdots\end{array}\\
\vdots & \vdots
\end{bmatrix}=T\{a_{i,j}\}$

In [3]:
#Escoger que archivo se desea normalizar los pesos
#chosen_file = "Productividad_Plataformas_Comerciales"
#chosen_file = "Productividad_Grandes_Cuentas"
chosen_file = "Productividad_Soluciones_Negocio"
#chosen_file = "Productividad_Pymes"

# Dataframe de archivo comisiones pestaña "Leyenda" de la ruta de Reporte de Productividad
section_1 = fl.SectionObj(inifile,chosen_file,month)
section_1.setParameter('presetsheet','Leyenda')
loader1 = fl.LoadFileProcess(section_1)
pesospltfrs = loader1.loadFile()

# Dataframe de archivo comisiones pestaña "Comisionantes" de la ruta de Reporte de Productividad
section_2 = fl.SectionObj(inifile,chosen_file,month)
section_2.setParameter('presetsheet','Comisionantes')
loader2 = fl.LoadFileProcess(section_2)
comisionantespltfrs = loader2.loadFile()

#Obteniendo ruta del archivo de comisiones para ser accedida por XlWings
file = section_2.getParameter('filelist')[0]
logger.debug(file)

#Abriendo archivo de Excel y haciendo hoja 'comisionantes' activa
wb = xw.Book(file)
comis_sheet = wb.sheets('Comisionantes')
leyenda_sheet = wb.sheets('Leyenda')
#for sheet in wb.sheets:
#  if 'Leyenda' in sheet:
#    logger.debug("yes")

### Inmovilizando valores de porcentajes ponderados
#Obteniendo indice de la columna de porcentaje ponderado
ponderado_cindex = dfutils.getExcelColIndexFromDF(comisionantespltfrs, "PORCENTAJE_TOTAL_PONDERADO")
#Hallando ultima fila de la columna 'PORCENTAJE_TOTAL_PONDERADO'
lastrow = comis_sheet.api.Cells(65536, ponderado_cindex).End(xw.constants.Direction.xlUp).Row
#Copiando como valores las contenidos de la columna
comis_sheet.range((1, ponderado_cindex)).options(transpose=True).value = comis_sheet.range((1, ponderado_cindex), (lastrow,ponderado_cindex)).value

# Nombres clave de la hoja leyenda, siempre verificar que esto no cambie, en caso lo haga modificar la siguiente tupla. Porque de lo contrario el algoritmo dara errores / no funcionara como se espera.
COMIS_COLUMNS_NAMES = ("CAPTURA", "GESTIÓN", "DESARROLLO", "ITEM")

### Filtramos la data de la pestaña 'Leyenda' y nos quedamos con la sección de pesos únicamente en el dataframe pesos_df
pesos_df = pesospltfrs
pesos_df = pesos_df[pesos_df['ITEM'] != COMIS_COLUMNS_NAMES[3]]
pesos_df = pesos_df[~pesos_df['ITEM'].isnull()]
pesos_df = pesos_df[pesos_df['ITEM'] < 3000]
pesos_df = pesos_df[pesos_df['ITEM'] > 2000]

mat_result = []

for i in range(3):
    regexp = COMIS_COLUMNS_NAMES[i] + ".*"
    #Nos quedamos unicamente con las columnas que comienzan con el patron en 'regexp'
    area_df = pesos_df.filter(regex=regexp)
    #Guardamos las cabeceras
    headers = area_df.columns.values
    area_df = area_df.fillna(0) #fill empty spaces read as nan to zeros
    #Convertimos el dataframe en una matriz numpy (array representation)
    area_mat = area_df.as_matrix() #mxn
    #Operaciones de matrices
    sum_mat_1 = np.sum(area_mat, axis=1) #1xm, sum all the rows
    #Invertimos escalarmente el vector
    sum_mat_2 = 1 / sum_mat_1
    #Reemplazamos los NaN por ceros
    sum_mat = np.nan_to_num(sum_mat_2, copy=True)
    #Creamos una matriz diagonal, donde los elementos en la diagonal son las sumas de cada fila.
    diag_mat = np.diag(sum_mat) #mxm
    #Multiplicacion matricial entre la matriz diagonal y la matrix original
    t_mat = np.dot(diag_mat, area_mat) #mxm x mxn = mxn
    #Resultado buscado
    df_conv = pd.DataFrame(t_mat, columns=headers)
    mat_result.append(df_conv)

df_conv = pd.concat([mat_result[0], mat_result[1], mat_result[2]], axis=1)

# Creamos copia de seguridad de la tabla de pesos antes de modificarla.
wb.sheets.add('backup_pesos')
backup_pesos_sheet = wb.sheets('backup_pesos')
backup_pesos_sheet.range('A1').value = pesos_df

#Buscar numero de columna CAPTURA 1
col_pesos = pesospltfrs.columns.get_loc(COMIS_COLUMNS_NAMES[0] + "_1") + 1
#Buscar  numero de fila primera ocurrencia de ITEM
row_pesos = min(pesospltfrs.index[pesospltfrs['ITEM'] == COMIS_COLUMNS_NAMES[3]].tolist()) + 4
# Escribimos los nuevos pesos en el lugar de los antiguos.
leyenda_sheet.range(row_pesos, col_pesos).options(pd.DataFrame, index=False).value = df_conv





El tamaño de Productividad_Soluciones_Negocio es 63 registros
El tamaño de Productividad_Soluciones_Negocio es 33 registros


C:\Users\jsaenza\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\jsaenza\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in true_divide


ValueError: Sheet named 'backup_pesos' already present in workbook

**Generar Reporte Productividad**
---

In [7]:
main_month = '201905'
months = ['201901','201902','201903','201904','201905']
#Diccionarios de Dataframes de los resultados de comisiones de la gente que nos interesa por mes (índice de diccionario)
results = {}

for month in months:
    print(month)
    # Cargar Data Frame de DNIs de los comisionantes del Reporte de Productividad Final
    # Esto determinara el orden final de la planilla resultante.
    section_3 = fl.SectionObj(inifile,"Productividad_Final")
    loader3 = fl.LoadFileProcess(section_3)
    dnis_deseado = loader3.loadFile()

    #Obtenemos el padron de empleados para obtener el Jefe Directo
    section_4 = fl.SectionObj(inifile,"Padron_Empleados")
    loader4 = fl.LoadFileProcess(section_4)
    empleados_df = loader4.loadFile()
    empleados_df = empleados_df[["DNI","JEFE_DIRECTO"]]

    # Cargar Dataframe de archivos de pesos comisiones
    section_5 = fl.SectionObj(inifile,"Productividad_All_Files",month)
    loader5 = fl.LoadFileProcess(section_5)
    resultados_comis= loader5.loadFile()
    #resultados_comis = resultados_comis[["DNI","PORCENTAJE_TOTAL_PONDERADO","FACTOR_DE_PAGO","CAPTURA","GESTIÓN","DESARROLLO"]]
    #resultados_comis = resultados_comis[["GERENCIA2","DNI","NOMBRES","POSICIÓN","FECHA_DE_INGRESO","PORCENTAJE_TOTAL_PONDERADO","FACTOR_DE_PAGO","CAPTURA","GESTIÓN","DESARROLLO"]]

    # Cruzamos con DNI del archivo de R. Productividad final
    resultados_de_planilla_deseada = pd.merge(dnis_deseado, resultados_comis, on='DNI', how='left')
    # Cruzamos con Ingresos para obtener el JEFE DIRECTO.
    resultados_de_planilla_deseada = pd.merge(resultados_de_planilla_deseada, empleados_df, on='DNI', how='left')
    #Eliminamos duplicados, pueden darse porque ftocci por ejemplo tiene 2 dnis, aunque ya se quito pero de todas maneras.
    resultados_de_planilla_deseada.drop_duplicates('DNI', inplace = True)
    #resultados_de_planilla_deseada = resultados_de_planilla_deseada[["DNI","PORCENTAJE_TOTAL_PONDERADO","FACTOR_DE_PAGO","CAPTURA","GESTIÓN","DESARROLLO"]]
    #if month == main_month:
    #    resultados_de_planilla_deseada = resultados_de_planilla_deseada[["GERENCIA2","DNI","NOMBRES","POSICIÓN","FECHA_DE_INGRESO","JEFE_DIRECTO","PORCENTAJE_TOTAL_PONDERADO","FACTOR_DE_PAGO","CAPTURA","GESTIÓN","DESARROLLO"]]
    #else
    #    resultados_de_planilla_deseada = resultados_de_planilla_deseada[["DNI","PORCENTAJE_TOTAL_PONDERADO","FACTOR_DE_PAGO","CAPTURA","GESTIÓN","DESARROLLO"]]
    
    results[month] = resultados_de_planilla_deseada


201901
El tamaño de Productividad_Final es 1 registros
El tamaño de Padron_Empleados es 2838 registros
El tamaño de Productividad_All_Files es 463 registros
201902
El tamaño de Productividad_Final es 1 registros
El tamaño de Padron_Empleados es 2838 registros
El tamaño de Productividad_All_Files es 461 registros
201903
El tamaño de Productividad_Final es 1 registros
El tamaño de Padron_Empleados es 2838 registros
El tamaño de Productividad_All_Files es 454 registros
201904
El tamaño de Productividad_Final es 1 registros
El tamaño de Padron_Empleados es 2838 registros
El tamaño de Productividad_All_Files es 451 registros
201905
El tamaño de Productividad_Final es 1 registros
El tamaño de Padron_Empleados es 2838 registros
El tamaño de Productividad_All_Files es 476 registros


In [7]:
results['201901']

,DNI,GERENCIA2,NOMBRES,POSICIÓN,FECHA_DE_INGRESO,PORCENTAJE_TOTAL_PONDERADO,FACTOR_DE_PAGO,CAPTURA,GESTIÓN,DESARROLLO,JEFE_DIRECTO
0,07267351,VD PYMES,JORGE DENEGRI,GERENTE DE CANAL DE VENTAS DIRECTAS PYMES,2015-08-01,0.7184,0.640000,0.718494,0.000000,0.000000,"FERNANDEZ RIVERA, JAVIER ELEODORO"
1,43053421,VD PYMES,AUGUSTO WONG BRUNO GASPAR SHIULONG,GERENTE DE NEGOCIOS VENTAS DIRECTAS PYMES,2018-10-01,0.6157,0.520000,0.769737,0.000000,0.000000,"DENEGRI AGUILAR, JORGE ENRIQUE"
2,10802141,VD PYMES,PAOLA SCHANKS OVALLE,GERENTE DE NEGOCIOS VENTAS DIRECTAS PYMES,2017-12-01,0.9705,0.950000,0.998916,0.000000,0.857143,"DENEGRI AGUILAR, JORGE ENRIQUE"
3,41327723,VD PYMES,GINO AURELIO BACIGALUPO SILVA,GERENTE DE NEGOCIOS VENTAS DIRECTAS PYMES,2017-02-01,0.0000,0.000000,0.000000,0.000000,0.000000,"DENEGRI AGUILAR, JORGE ENRIQUE"
4,41877014,CORPORACIONES,MARIA ISABEL SANTA CRUZ FUENTES,EJECUTIVO DE CORPORACIONES,2014-07-01,3.1321,3.472697,3.685980,1.282283,4.000000,"DENEGRI AGUILAR, JORGE ENRIQUE"
5,40668745,VD PYMES,CESAR GUILLERMO RIOS CHAVARRI,GERENTE DE NEGOCIOS VENTAS DIRECTAS PYMES,2007-07-01,0.8126,0.760000,0.837278,0.000000,0.714286,"DENEGRI AGUILAR, JORGE ENRIQUE"
6,46721859,VD PYMES,MARISOL MARTINEZ TERAN,CONSULTOR VD PYMES,2017-03-01,2.4642,3.452667,2.464271,0.000000,0.000000,"AUGUSTO WONG, BRUNO GASPAR SHIULONG"
7,000235914,VD PYMES,MARTIN ALEJANDRO SOTO VILLARREAL,CONSULTOR VD PYMES,2018-11-01,2.0059,2.540000,2.005969,0.000000,0.000000,"AUGUSTO WONG, BRUNO GASPAR SHIULONG"
8,45508130,VD PYMES,ROBERTO GONZALO OJEDA TOVAR,CONSULTOR VD PYMES,2017-09-16,0.7583,0.538123,0.758376,0.000000,0.000000,"AUGUSTO WONG, BRUNO GASPAR SHIULONG"
9,70026053,VD PYMES,TATSUMI AMARILIS TAKAMURA ARBOLEDA,CONSULTOR VD PYMES,2017-10-01,0.0000,0.000000,0.000000,0.000000,0.000000,"AUGUSTO WONG, BRUNO GASPAR SHIULONG"


In [8]:
final_df_ponderados = results[main_month]
final_df_ponderados = final_df_ponderados[["GERENCIA2","DNI","NOMBRES","POSICIÓN","FECHA_DE_INGRESO","FECHA_CESE","JEFE_DIRECTO"]]
for month in months:
    df1 = results[month]
    df1 = df1.rename(columns={"PORCENTAJE_TOTAL_PONDERADO": month})
    final_df_ponderados = pd.concat([final_df_ponderados, df1[month]], axis=1)

final_df_cajas = results[main_month]
final_df_cajas = final_df_cajas[["GERENCIA2","DNI","NOMBRES","POSICIÓN","FECHA_DE_INGRESO","FECHA_CESE","JEFE_DIRECTO"]]
for month in months:
    df2 = results[month]
    df2 = df2.rename(columns={"CAPTURA": month+"_CAPTURA" , "GESTIÓN": month+"_GESTIÓN", "DESARROLLO": month+"_DESARROLLO"})
    final_df_cajas = pd.concat([final_df_cajas, df2[month+"_CAPTURA"],df2[month+"_GESTIÓN"],df2[month+"_DESARROLLO"]], axis=1)
    


In [29]:
final_df_ponderados

,GERENCIA2,DNI,NOMBRES,POSICIÓN,FECHA_DE_INGRESO,JEFE_DIRECTO
0,CORPORACIONES,25720995,PAUL ANTONIO DIAZ NOZIGLIA,GERENTE DE NEGOCIOS CARTERA CORPORACIONES,2003-06-01,"CÁCERES UCEDA, EDUARDO ESTEBAN"
1,CORPORACIONES,40134270,ALVARO FERNANDO GOMEZ DE CORDOVA CALLIRGOS,GERENTE DE NEGOCIOS CARTERA CORPORACIONES,2007-05-01,"CÁCERES UCEDA, EDUARDO ESTEBAN"
2,CORPORACIONES,40003514,ROMINA MESETH MACCHIAVELLO,GERENTE DE NEGOCIOS CARTERA CORPORACIONES,2002-09-01,"CÁCERES UCEDA, EDUARDO ESTEBAN"
3,CORPORACIONES,10542531,FRANCISCO TOCCI VAN OORDT,GERENTE DE NEGOCIOS CARTERA CORPORACIONES,2017-02-01,"CÁCERES UCEDA, EDUARDO ESTEBAN"
4,GRANDES CLIENTES,41131333,EDUARDO GOICOCHEA ORTIZ DE ZEVALLOS,GERENTE DE NEGOCIOS CARTERA GRANDES CLIENTES,2003-12-01,"SUAREZ ASCARZA, ALAIN"
5,GRANDES CLIENTES,42244365,JORGE NARUSE BURGA,GERENTE DE NEGOCIOS CARTERA GRANDES CLIENTES,2016-09-01,"SUAREZ ASCARZA, ALAIN"
6,GRANDES CLIENTES,41812901,GERARDO DANIEL MUÑOZ JUGO,GERENTE DE NEGOCIOS CARTERA GRANDES CLIENTES,2017-02-01,"SUAREZ ASCARZA, ALAIN"
7,GRANDES CLIENTES,40197601,SANDRA JIMENO GOICOCHEA,GERENTE DE NEGOCIOS CARTERA GRANDES CLIENTES,2003-02-01,"SUAREZ ASCARZA, ALAIN"
8,VENTA REGIONAL EMPRESA,40908487,LUIS ALBERTO CHACON REBAZA,GERENTE DE NEGOCIOS REGIONAL EMPRESAS,2007-05-01,"FERNANDEZ RIVERA, JAVIER ELEODORO"
9,VENTA REGIONAL EMPRESA,40219881,MAURICIO GUERRA PONCE,GERENTE DE NEGOCIOS REGIONAL EMPRESAS,2016-06-01,"FERNANDEZ RIVERA, JAVIER ELEODORO"


In [9]:
dataoutdir = inifile.getTestPath()
writer = pd.ExcelWriter(posixpath.join(dataoutdir,"ReporteProdutividad_j.xlsx"), engine='xlsxwriter')
final_df_ponderados.to_excel(writer, sheet_name='ponderados')
final_df_cajas.to_excel(writer, sheet_name='cajas')
writer.save()

